# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [1]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-04-04 18:12:07--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M   169MB/s    in 0.2s    

2023-04-04 18:12:08 (169 MB/s) - ‘train.csv’ saved [29470338/29470338]



,label,title,lead,text
45454,sci/tech,Recording Industry Sues 762 for Net Music Swap...,Reuters - A recording-industry trade group\sai...,Recording Industry Sues 762 for Net Music Swap...
1203,sport,"Hewitt, Sluiter, Muller Win at Legg Mason (AP)","AP - Lleyton Hewitt breezed to a 6-1, 6-2 win ...","Hewitt, Sluiter, Muller Win at Legg Mason (AP)..."
112127,business,Eurostocks open higher,PARIS: European shares opened higher on Friday...,Eurostocks open higher PARIS: European shares ...
54600,sci/tech,"HP, Microsoft Take Tech on the Road (Ziff Davis)","Ziff Davis - At TechXNY Expo, the duo showcase...","HP, Microsoft Take Tech on the Road (Ziff Davi..."
101844,sport,Bulldog victory closes Manning Bowl,The final Manning Bowl bragging rights belong ...,Bulldog victory closes Manning Bowl The final ...


## Hugginface Transformers

In [3]:
#!pip install transformers
from transformers import DistilBertForSequenceClassification, DistilBertConfig
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.7 MB/s eta 0:00:00


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##TODO build a transformer model to do sequence classification with the goal to predict the label from the text

model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder 
model = DistilBertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [5]:
##TODO print the summary of the model

print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
from sklearn.model_selection import train_test_split

##TODO split the sample into a training and a test set 

label2id = {"world": 0, "sport": 1, "business": 2, "sci/tech": 3}
y = df["label"].apply(lambda x: label2id[x]).values

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), y.tolist(), test_size = .2)

##TODO prepare the dataset for torch.

model.to(device)


<class 'list'>


In [ ]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. To learn more, checkout the trainer class of huggingface transformers)

# Doc Embedding

In [ ]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

In [ ]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

In [ ]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

In [ ]:
##TODO train Doc2Vec on the texts in the dataset
##TODO derive the word vectors for each text in the dataset
##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

In [ ]:
##TODO do the same with embeddings provided by spaCy

In [ ]:
##TODO do the same with SBERT embeddings